In [99]:
import bs4
import requests

def get_links(start):
    res = requests.get(start)    
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    links = []
    
    for link in soup.find_all('a', href=True):        
        # apenas links do site
        if (link["href"].startswith(start) | link["href"].startswith("/")):
            links.append(link["href"])
            print(link["href"]) # conferindo
            
    return links
    

def crawl_bsf(start):    
    links = get_links(start)


In [102]:
'''
seeds = ["https://www.minhavida.com.br/saude", "https://minutosaudavel.com.br", "https://medicoresponde.com.br", 
             "https://emagrecer.eco.br", "https://www.tuasaude.com", "https://www.mdsaude.com", 
             "http://portalms.saude.gov.br", "http://www.riocomsaude.rj.gov.br", "http://www.portalnatural.com", 
             "http://portal.saude.pe.gov.br"]
'''
seeds = ["https://www.minhavida.com.br"]

for link in seeds:
    crawl_bsf(link)

/
/saude
/saude/temas
/saude/bulas
/guias/dor-nas-costas
/guias/saude-bucal
/guias/convivendo-com-artrite-psoriasica
/guias/dii
/guias/especial-medley
/guias/especial-autocuidado
/guias/diarreia
/guias/eu-de-lentes
/alimentacao
/alimentacao/tudo-sobre
/receita.htm
/guias/especial-vigor
/guias/o-poder-das-sementes
/beleza
/beleza/tudo-sobre
/fitness
/familia
/especiais/obesidade-infantil
/guias/puberdade-precoce
/guias/colica-do-bebe
/bem-estar
/
/saude/noticias/33832-ziraldo-e-internado-em-estado-grave-apos-sofrer-avc
/saude/noticias/33832-ziraldo-e-internado-em-estado-grave-apos-sofrer-avc
/saude/noticias/33832-ziraldo-e-internado-em-estado-grave-apos-sofrer-avc
/saude/noticias/33828-estudo-investiga-uso-de-canabidiol-para-tratamento-contra-depressao
/saude/noticias/33828-estudo-investiga-uso-de-canabidiol-para-tratamento-contra-depressao
/saude/noticias/33828-estudo-investiga-uso-de-canabidiol-para-tratamento-contra-depressao
/saude/noticias/33825-ciro-gomes-e-internado-para-realizar

https://emagrecer.eco.br/
https://emagrecer.eco.br/como-perder-barriga/
https://emagrecer.eco.br/dieta-e-cardapio/
https://emagrecer.eco.br/cha/
https://emagrecer.eco.br/suco/
https://emagrecer.eco.br/sopa/
https://emagrecer.eco.br/remedio/
https://emagrecer.eco.br/beleza/
https://emagrecer.eco.br/alimento/
https://emagrecer.eco.br/saude/
https://emagrecer.eco.br/remedio/
https://emagrecer.eco.br/como-perder-barriga/
https://emagrecer.eco.br/dieta-e-cardapio/
https://emagrecer.eco.br/suco/
https://emagrecer.eco.br/cha/
https://emagrecer.eco.br/sopa/
https://emagrecer.eco.br/alimento/
https://emagrecer.eco.br/como-emagrecer-rapido/
https://emagrecer.eco.br/como-emagrecer-rapido/em-uma-semana-3-5kg-10-quilos/
https://emagrecer.eco.br/como-emagrecer-rapido/emagrecimento-dietas-cardapio-receitas/
https://emagrecer.eco.br/como-emagrecer-rapido/com-saude-dietas-cardapio-receitas/
https://emagrecer.eco.br/como-emagrecer-rapido/urgente-preciso-quero-10-20kgs-em-uma-semana-3-dias/
https://emagr

/acessibilidade
/mapa-do-site
http://portalms.saude.gov.br/
/fale-conosco
/comunicacao-e-imprensa
/assessoria-de-imprensa
/acoes-e-programas/vacinacao/vacine-se
/institucional/o-ministerio
/acoes-e-programas
/sistema-unico-de-saude
/noticias
/atencao-especializada-e-hospitalar
/atencao-basica
/assistencia-farmaceutica
/ciencia-e-tecnologia-e-complexo-industrial
/gestao-do-sus
/vigilancia-em-saude
/participacao-e-controle-social
/repasses-financeiros
/saude-para-voce
/saude-indigena
/saude-de-a-z
/trabalho-educacao-e-qualificacao
/politica-nacional-de-saude-mental-alcool-e-outras-drogas
/dados-e-indicadores-da-saude
/boletins-epidemiologicos
/protocolos-e-diretrizes
/institucional/o-ministerio
/acoes-e-programas
/agenda-das-autoridades
/auditorias
/audiencias-e-consultas-publicas
/concursos-e-selecoes
/convenios-e-repasses
/despesas
/informacoes-classificadas
/legislacao-da-saude
/licitacoes-e-contratos
/servidores
/servico-de-informacao-ao-cidadao-sic
/relatorio-de-gestao
/sei
/aplicat

/secretaria
/secretaria
/organograma
/secretario
/secretaria-executiva/secretaria-executiva-de-coordenacao-geral
/secretaria-executiva/secretaria-executiva-de-atencao-saude
/secretaria-executiva/secretaria-executiva-de-vigilancia-em-saude
/secretaria-executiva/secretaria-executiva-de-administracao-e-financas
/secretaria-executiva/secretaria-executiva-de-regulacao-em-saude
/secretaria-executiva/secretaria-executiva-de-gestao-do-trabalho-e-educacao-em-saude
/superintendencia-de-comunicacao
/secretaria/gerencia-geral-de-assuntos-juridicos
/orgaos-colegiados
/unidades-de-saude-e-servicos/secretaria/ouvidoria
/unidades-servicos
/hospitais
/upas
/unidades-de-saude-e-servicos/secretaria-executiva-de-atencao-saude/upaes
/unidades-de-saude-e-servicos/secretaria-executiva-de-vigilancia-em-saude/apevisa
/unidades-de-saude-e-servicos/secretaria-executiva-de-atencao-saude/ceatox
/unidades-de-saude-e-servicos/secretaria-executiva-de-regulacao-em-saude/central-de-transplantes
/unidades-de-saude-e-ser

## Tarefas  

<input type="checkbox" disabled checked> crawlear
<input type="checkbox" disabled checked> tratar links que não são do site
<input type="checkbox"> tratar links relativos
<input type="checkbox"> BFS
<input type="checkbox"> sleep - não sobrecarregar
<input type="checkbox"> tratar links iguais (ideal seria set(), mas não preserva a ordem de inserção) ?
<input type="checkbox"> respeitar robots.txt ?